In [ ]:
HTML(read(open("style.html"), String))

In [ ]:
using Pkg
Pkg.add("NBInclude")

In [16]:
using NBInclude
@nbinclude("3.0 Piece-Weight & Value calculation.ipynb")
@nbinclude("3.1 Simple Evaluation Function.ipynb")

   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`


evaluate_for_color (generic function with 1 method)

***

## Transpositions-Tabellen

Dieses Notebook enthält den [MiniMax-Algorithmus](https://en.wikipedia.org/wiki/Minimax) mit einer [Transpositions-Tabellen](https://de.wikibrief.org/wiki/Transposition_table)-Erweiterung.

Die Grundidee der Transpositions-Tabellen-Optimierung ist, dass unterschiedliche Zugfolgen das gleiche Spielbrett erzeugen können. Die zugrundeliegende, tieferen Suchbäume sind somit gleich.

Wird ein Suchbaum evaluiert, so wird das Ergebnis der Suche abgespeichert, sodass bei einem erneuten Besuchen des gleichen Baumes der Wert direkt abgerufen werden kann, ohne den Baum neu evaluieren zu müssen (Caching).

Eine effiziente Realisierung des Cache-Vorgangs ist eine Hashtabelle (Dictionary in Julia). Das resultierende Schachbrett wird als Schlüssel benutzt.

In [31]:
function minimax_transpositions(score_white, score_black, board, requiredDepth)

    transpositions = Dict(""=>0)
    best_move = nothing
    
    function minimax(score_white, score_black, board, depth)
        
        # check if we already processed this board stage
        board_fen = fen(board)
        maxValue = get(transpositions, board_fen, nothing) # nothing as default value (instead of throwing an exception)
        
        # if maxValue != nothing, we can proceed
        
        # no? -> process as before and set values
        if maxValue === nothing
            possibleMoves = moves(board)
            if depth == 0 || size(possibleMoves) == 0
                if sidetomove(board) == WHITE
                    return score_white - score_black
                else
                    return score_black - score_white
                end
            end

            maxValue = 1 << 63;
            for move in possibleMoves

                new_score_white, new_score_black = evaluate_incremental(score_white, score_black, board, move)
                undoInfo = domove!(board, move)
                value = -minimax(new_score_white, new_score_black, board, depth - 1)
                undomove!(board, undoInfo)

                if value > maxValue
                    maxValue = value
                    if depth == requiredDepth
                        println("Best move: ", move, " score ", value)
                        best_move = move
                    end
                end
            end
            
            # add value to table
            transpositions[board_fen] = maxValue
            recycle!(possibleMoves)
        end
        
        return maxValue
    end
    
    minimax(score_white, score_black, board, requiredDepth)
    return best_move
end

minimax_transpositions (generic function with 1 method)

Testen wir diese Generation der AI gegen die vorherige Version (MiniMaxProgressiveIncAI), so ist eine leichte Verschlechterung der Performance zu erkennen. 

Der Grund hierfür ist die verwendete Hashing Methode (Abbildung Schachbrett → Schlüssel für Hashtabelle). Aktuell verwenden wir die Funktion `fen(board)`, um das Schachbrett in einen eindeutigen FEN-String zu konvertieren. Dieser wird anschließend von Julia auf einen Hashwert abgebildet. Während Julias String-Hashing sehr schnell verläuft, benötigt das Konvertieren des Schachbretts in einen FEN-String eine spürbare Zeit.

Dieses Problem können wir beheben, indem wir das Schachbrett anderweitig auf einen Hashwert abbilden. Hierfür können wir [Zobrist Hashing](https://en.wikipedia.org/wiki/Zobrist_hashing) verwenden (TODO).

***

In [32]:
using NBInclude
@nbinclude("1.2 BasicGame.ipynb")
@nbinclude("2.1 MiniMaxAI.ipynb")

   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to 

next_move (generic function with 5 methods)

In [33]:
play_game(MiniMaxTranspositionsAI(4, 4))

Board (r1bqkbnr/pppppppp/8/4n3/2B1P3/8/PPPP1PPP/RNBQK1NR w KQkq -):
 r  -  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  n  -  -  - 
 -  -  B  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  -  N  R